In [2]:
import numpy as np
import yaml

mass_vals = np.zeros((2,))
mass_vals[1]

np.float64(0.0)

In [5]:
def load_inertia_params():
    
    mass_vals = np.zeros((6,))
    inertia_vals = []
    with open('./config/xarm6_inertia.yaml', 'r') as file:
        inertia_params = yaml.safe_load(file)
        for i in range(0,6):
            link = 'link' + str(i+1)
            inertia = inertia_params[link]['inertia']
            inertia_vals.append(
                np.array(
                    [[inertia['ixx'], -inertia['ixy'], -inertia['ixz']],
                     [0, inertia['iyy'], -inertia['iyz']],
                     [0, 0, inertia['izz']]]
                )
            )
            mass_vals[i] = inertia_params[link]['mass']

        return mass_vals, inertia_vals
            
mass, inertia = load_inertia_params()
mass, inertia

(array([2.3814 , 2.2675 , 1.875  , 1.3192 , 1.33854, 0.17   ]),
 [array([[ 5.856200e-03,  1.790000e-05, -3.550000e-06],
         [ 0.000000e+00,  5.031600e-03,  8.883360e-04],
         [ 0.000000e+00,  0.000000e+00,  3.536652e-03]]),
  array([[ 0.02831578, -0.005     , -0.00066546],
         [ 0.        ,  0.0058    ,  0.0045741 ],
         [ 0.        ,  0.        ,  0.0273447 ]]),
  array([[ 0.0063483 ,  0.0015397 , -0.00096858],
         [ 0.        ,  0.00379758, -0.00186567],
         [ 0.        ,  0.        ,  0.00595768]]),
  array([[4.89600e-03, 6.92500e-06, 1.41800e-05],
         [0.00000e+00, 4.45694e-03, 2.31860e-04],
         [0.00000e+00, 0.00000e+00, 1.34332e-03]]),
  array([[ 0.00146378,  0.00045062, -0.00028431],
         [ 0.        ,  0.00184192, -0.00013087],
         [ 0.        ,  0.        ,  0.00233352]]),
  array([[9.30e-05, 0.00e+00, 0.00e+00],
         [0.00e+00, 5.87e-05, 3.60e-06],
         [0.00e+00, 0.00e+00, 1.32e-04]])])

In [6]:
arr = np.zeros((6,4,4))
arr[0]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [7]:
arr[0] = [[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]

In [42]:
from sympy import Symbol, Matrix
_q1 = Symbol('q1')
_q2 = Symbol('q2')
_q1, _q2

(q1, q2)

In [261]:
type(cos(np.pi))

sympy.core.numbers.Float

In [44]:
X = Matrix([_q1, _q1*_q2, 0, _q1])
Y = Matrix([_q1, _q2])
X,Y

(Matrix([
 [   q1],
 [q1*q2],
 [    0],
 [   q1]]),
 Matrix([
 [q1],
 [q2]]))

In [46]:
X.jacobian(Y)

Matrix([
[ 1,  0],
[q2, q1],
[ 0,  0],
[ 1,  0]])

In [24]:
from robot import Robot
from sympy import sin, cos

robot = Robot('192.168.1.223')

ROBOT_IP: 192.168.1.223, VERSION: v1.12.10, PROTOCOL: V1, DETAIL: 6,6,XI1304,AC1303,v1.12.10, TYPE1300: [1, 1]
change protocol identifier to 3
[SDK][ERROR][2025-03-19 11:25:38][base.py:380] - - API -> set_tcp_load -> code=9, weight=0.028, center=[0.0, 0.0, 0.0]
[set_state], xArm is ready to move


ControllerError, code: 1
ControllerError had clean


In [48]:
q0 = Symbol('q0')
q1 = Symbol('q1')
q2 = Symbol('q2')
q3 = Symbol('q3')
q4 = Symbol('q4')
q5 = Symbol('q5')
q = Matrix([q0, q1, q2, q3, q4, q5])
q

Matrix([
[q0],
[q1],
[q2],
[q3],
[q4],
[q5]])

In [104]:
T = Matrix([0, 0, 0.267])
jV = T.jacobian(Matrix([q0, q1, q2, q3, q4, q5]))
jW = Matrix([
    [sin(q1+robot.dh_params[0][0].item())*sin(robot.dh_params[0][2].item()), 0,0,0,0,0],
    [-cos(q1+robot.dh_params[0][0].item())*sin(robot.dh_params[0][2].item()), 0,0,0,0,0],
    [cos(robot.dh_params[0][2].item()),0,0,0,0,0]])
J = Matrix([jV, jW])
J

Matrix([
[                         0, 0, 0, 0, 0, 0],
[                         0, 0, 0, 0, 0, 0],
[                         0, 0, 0, 0, 0, 0],
[-0.999999999999999*sin(q1), 0, 0, 0, 0, 0],
[ 0.999999999999999*cos(q1), 0, 0, 0, 0, 0],
[      -4.37113900018624e-8, 0, 0, 0, 0, 0]])

Looks like if we want to calculate the jacobian $\mathbf{J}$, the best way would be to iterate over the various transformation matrices in the SymPy library, multiply them together and then compute the specific twists for each joint. Then we can create the body Jacobian.

In [97]:
T = Matrix(4,4, [cos(q1+robot.dh_params[0][0].item()), -sin(q1+robot.dh_params[0][0].item())*cos(robot.dh_params[0][2].item()), sin(q1+robot.dh_params[0][0].item())*sin(robot.dh_params[0][2].item()), robot.dh_params[0][3].item()*cos(q1+robot.dh_params[0][0].item()),
        sin(q1+robot.dh_params[0][0].item()), cos(q1+robot.dh_params[0][0].item())*cos(robot.dh_params[0][2].item()), -cos(q1+robot.dh_params[0][0].item())*sin(robot.dh_params[0][2].item()), robot.dh_params[0][3].item()*sin(q1+robot.dh_params[0][0].item()),
           0, sin(robot.dh_params[0][2].item()), cos(robot.dh_params[0][2].item()), robot.dh_params[0][1].item(),
           0, 0, 0, 1])
T

Matrix([
[cos(q1),  4.37113900018624e-8*sin(q1), -0.999999999999999*sin(q1),                 0],
[sin(q1), -4.37113900018624e-8*cos(q1),  0.999999999999999*cos(q1),                 0],
[      0,           -0.999999999999999,       -4.37113900018624e-8, 0.266999989748001],
[      0,                            0,                          0,                 1]])

In [226]:
robot.inertia[0].shape

(3, 3)

In [256]:
rot_axis = [2, 1, 1, 2, 1, 2]

T = sympy.zeros(4,4)

J = sympy.zeros(6,6)
M = sympy.zeros(6,6)

for i in range(0, 6):
    if i == 0:
        T = Matrix(4,4,[
            cos(q[i]+robot.dh_params[i][0].item()), -sin(q[i]+robot.dh_params[i][0].item())*cos(robot.dh_params[i][2].item()), sin(q[i]+robot.dh_params[i][0].item())*sin(robot.dh_params[i][2].item()), robot.dh_params[i][3].item()*cos(q[i]+robot.dh_params[i][0].item()),
            sin(q[i]+robot.dh_params[i][0].item()), cos(q[i]+robot.dh_params[i][0].item())*cos(robot.dh_params[i][2].item()), -cos(q[i]+robot.dh_params[i][0].item())*sin(robot.dh_params[i][2].item()), robot.dh_params[i][3].item()*sin(q[i]+robot.dh_params[i][0].item()),
            0, sin(robot.dh_params[i][2].item()), cos(robot.dh_params[i][2].item()), robot.dh_params[i][1].item(),
            0, 0, 0, 1
        ])
        T_p = T[0:3,3]
        jV = T_p.jacobian(q)
        jW = Matrix([[T[0:3,rot_axis[i]]]])
        J[0:3,:] = jV
        J[3:6,i] = jW
        M = robot.mass[i].item()*jV.T*jV + J[3:6,:].T*Matrix(robot.inertia[i])*J[3:6,:]
    else:
        T_temp = Matrix(4,4,[
            cos(q[i]+robot.dh_params[i][0].item()), -sin(q[i]+robot.dh_params[i][0].item())*cos(robot.dh_params[i][2].item()), sin(q[i]+robot.dh_params[i][0].item())*sin(robot.dh_params[i][2].item()), robot.dh_params[i][3].item()*cos(q[i]+robot.dh_params[i][0].item()),
            sin(q[i]+robot.dh_params[i][0].item()), cos(q[i]+robot.dh_params[i][0].item())*cos(robot.dh_params[i][2].item()), -cos(q[i]+robot.dh_params[i][0].item())*sin(robot.dh_params[i][2].item()), robot.dh_params[i][3].item()*sin(q[i]+robot.dh_params[i][0].item()),
            0, sin(robot.dh_params[i][2].item()), cos(robot.dh_params[i][2].item()), robot.dh_params[i][1].item(),
            0, 0, 0, 1
        ])
        T = T * T_temp
        T_p = T[0:3,3]
        jV = T_p.jacobian(q)
        jW = Matrix([[T[0:3,rot_axis[i]]]])
        J[0:3,:] = jV
        J[3:6,i] = jW
        M = robot.mass[i].item()*jV.T*jV + J[3:6,:].T*Matrix(robot.inertia[i])*J[3:6,:]

M.subs(eval).evalf(3)

Matrix([
[ 0.00737, -6.66e-7,  -5.87e-5,  -0.00262,  -5.87e-5,  -3.6e-6],
[-1.01e-9,   0.0115,    0.0171, -1.17e-10,   0.00525,  2.44e-5],
[-5.87e-5,   0.0171,     0.037,  -5.87e-5,    0.0093,   3.6e-6],
[-0.00262, -6.65e-7,  -5.87e-5,   0.00104,  -5.87e-5,  -3.6e-6],
[-5.87e-5,  0.00525,    0.0093,  -5.87e-5,   0.00264,   3.6e-6],
[6.38e-13,  2.44e-5, -6.41e-12,  6.38e-13, -6.41e-12, 0.000132]])

In [257]:
eval = {
    q[0]: 0,
    q[1]: 0,
    q[2]: 0,
    q[3]: 0,
    q[4]: 0,
    q[5]: 0
}
M_eval = M.subs(eval)
M_eval = M_eval.evalf(n=3)
M_eval

Matrix([
[ 0.00737, -6.66e-7,  -5.87e-5,  -0.00262,  -5.87e-5,  -3.6e-6],
[-1.01e-9,   0.0115,    0.0171, -1.17e-10,   0.00525,  2.44e-5],
[-5.87e-5,   0.0171,     0.037,  -5.87e-5,    0.0093,   3.6e-6],
[-0.00262, -6.65e-7,  -5.87e-5,   0.00104,  -5.87e-5,  -3.6e-6],
[-5.87e-5,  0.00525,    0.0093,  -5.87e-5,   0.00264,   3.6e-6],
[6.38e-13,  2.44e-5, -6.41e-12,  6.38e-13, -6.41e-12, 0.000132]])

In [258]:
M_eval.applyfunc(lambda i: 0 if -1e-3<i<1e-3 else i)

Matrix([
[ 0.00737,       0,      0, -0.00262,       0, 0],
[       0,  0.0115, 0.0171,        0, 0.00525, 0],
[       0,  0.0171,  0.037,        0,  0.0093, 0],
[-0.00262,       0,      0,  0.00104,       0, 0],
[       0, 0.00525, 0.0093,        0, 0.00264, 0],
[       0,       0,      0,        0,       0, 0]])

In [266]:
import jax.numpy as jnp

t = np.array(M_eval).astype(np.float64)
jnp.array(t)

Array([[ 7.3699951e-03, -6.6601206e-07, -5.8703125e-05, -2.6206970e-03,
        -5.8703125e-05, -3.6000274e-06],
       [-1.0097665e-09,  1.1541367e-02,  1.7097473e-02, -1.1652901e-10,
         5.2518845e-03,  2.4385750e-05],
       [-5.8703125e-05,  1.7097473e-02,  3.7002563e-02, -5.8703125e-05,
         9.3040466e-03,  3.6000274e-06],
       [-2.6206970e-03, -6.6519715e-07, -5.8703125e-05,  1.0406971e-03,
        -5.8703125e-05, -3.6000274e-06],
       [-5.8703125e-05,  5.2518845e-03,  9.3040466e-03, -5.8703125e-05,
         2.6402473e-03,  3.6000274e-06],
       [ 6.3815619e-13,  2.4385750e-05, -6.4082073e-12,  6.3815619e-13,
        -6.4082073e-12,  1.3199449e-04]], dtype=float32)

In [268]:
jnp.array(M_eval)

TypeError: Value '[[0.00737 -6.66e-7 -5.87e-5 -0.00262 -5.87e-5 -3.60e-6]
 [-1.01e-9 0.0115 0.0171 -1.17e-10 0.00525 2.44e-5]
 [-5.87e-5 0.0171 0.0370 -5.87e-5 0.00930 3.60e-6]
 [-0.00262 -6.65e-7 -5.87e-5 0.00104 -5.87e-5 -3.60e-6]
 [-5.87e-5 0.00525 0.00930 -5.87e-5 0.00264 3.60e-6]
 [6.38e-13 2.44e-5 -6.41e-12 6.38e-13 -6.41e-12 0.000132]]' with dtype object is not a valid JAX array type. Only arrays of numeric types are supported by JAX.

In [96]:
eval = {
    q[0]: 0,
    q[1]: 0,
    q[2]: 0,
    q[3]: 0,
    q[4]: 0,
    q[5]: 0
}
T.subs(eval)

Matrix([
[1.0, 2.86759601556699e-24, -2.77555756156289e-17,    0.207386745440213],
[  0,   -0.999999999999996,  -8.74227800037247e-8, -2.60508448676684e-8],
[  0,  8.74227800037247e-8,    -0.999999999999996,    0.111026149273597],
[  0,                    0,                     0,                    1]])

In [133]:
Matrix(robot.inertia[0])

Matrix([
[0.0058562,   1.79e-5,    -3.55e-6],
[        0, 0.0050316, 0.000888336],
[        0,         0,  0.00353665]])